# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ovookpubuluku/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ovookpubuluku/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Use-Case Data!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

/Users/ovookpubuluku/project-repos/ai-makerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/segmenter.py:66: SyntaxWarning: invalid escape sequence '\s'
  for match in re.finditer('{0}\s*'.format(re.escape(sent)), self.original_text):
/Users/ovookpubuluku/project-repos/ai-makerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/arabic.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)
/Users/ovookpubuluku/project-repos/ai-makerspace/AIE8/07_Synthetic_Data_Generation_and_LangSmith/.venv/lib/python3.13/site-packages/pysbd/lang/persian.py:29: SyntaxWarning: invalid escape sequence '\.'
  txt = re.sub('(?<={0})\.'.format(am), '∯', txt)


Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 64, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node '21ba03'. Skipping!
Property 'summary' already exists in node '941650'. Skipping!
Property 'summary' already exists in node '57e602'. Skipping!
Property 'summary' already exists in node 'aaf254'. Skipping!
Property 'summary' already exists in node '0795f2'. Skipping!
Property 'summary' already exists in node 'a7f0b7'. Skipping!
Property 'summary' already exists in node 'ec4da7'. Skipping!
Property 'summary' already exists in node '28a046'. Skipping!
Property 'summary' already exists in node '4578b4'. Skipping!
Property 'summary' already exists in node '2f7a5c'. Skipping!
Property 'summary' already exists in node 'a4fb39'. Skipping!
Property 'summary' already exists in node '837a3d'. Skipping!
Property 'summary' already exists in node 'de57fd'. Skipping!
Property 'summary' already exists in node 'cbbf46'. Skipping!
Property 'summary' already exists in node '5d6ef4'. Skipping!
Property 'summary' already exists in node '599cc0'. Skipping!
Property

Applying CustomNodeFilter:   0%|          | 0/8 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '28a046'. Skipping!
Property 'summary_embedding' already exists in node '21ba03'. Skipping!
Property 'summary_embedding' already exists in node 'ec4da7'. Skipping!
Property 'summary_embedding' already exists in node '57e602'. Skipping!
Property 'summary_embedding' already exists in node '941650'. Skipping!
Property 'summary_embedding' already exists in node 'a7f0b7'. Skipping!
Property 'summary_embedding' already exists in node '837a3d'. Skipping!
Property 'summary_embedding' already exists in node 'cbbf46'. Skipping!
Property 'summary_embedding' already exists in node 'a4fb39'. Skipping!
Property 'summary_embedding' already exists in node '0795f2'. Skipping!
Property 'summary_embedding' already exists in node '2f7a5c'. Skipping!
Property 'summary_embedding' already exists in node 'aaf254'. Skipping!
Property 'summary_embedding' already exists in node 'de57fd'. Skipping!
Property 'summary_embedding' already exists in node '5d6ef4'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 86, relationships: 711)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("usecase_data_kg.json")
usecase_data_kg = KnowledgeGraph.load("usecase_data_kg.json")
usecase_data_kg

KnowledgeGraph(nodes: 86, relationships: 711)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=usecase_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,How many messages are sent weekly with 18 bill...,[Introduction ChatGPT launched in November 202...,"By July 2025, 18 billion messages were being s...",single_hop_specifc_query_synthesizer
1,How is the usage of ChatGPT different from oth...,[Table 1: ChatGPT daily message counts (millio...,"According to the provided context, ChatGPT's u...",single_hop_specifc_query_synthesizer
2,What does the context say about the role of en...,[Variation by Occupation Figure 23 presents va...,The context indicates that users in engineerin...,single_hop_specifc_query_synthesizer
3,Whay are the relashionships between user demog...,[Conclusion This paper studies the rapid growt...,The context indicates that gender gaps in Chat...,single_hop_specifc_query_synthesizer
4,Based on the data from June 2024 and June 2025...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data shows that from June 2024 to June 202...,multi_hop_abstract_query_synthesizer
5,how ChatGPT expanding in low middle countries ...,[<1-hop>\n\nConclusion This paper studies the ...,The context shows that ChatGPT usage has grown...,multi_hop_abstract_query_synthesizer
6,Based on the data showing that non-work messag...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,The data indicates that from June 2024 to June...,multi_hop_abstract_query_synthesizer
7,Based on the analysis of ChatGPT's rapid adopt...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,The context indicates that since the launch of...,multi_hop_specific_query_synthesizer
8,OpenAI ChatGPT use info and how much work and ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"ChatGPT launched in November 2022, and by July...",multi_hop_specific_query_synthesizer
9,How do Brynjolfsson's findings relate to the r...,[<1-hop>\n\nConclusion This paper studies the ...,"Brynjolfsson's research, which is referenced t...",multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/21 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/64 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to ap

Applying SummaryExtractor:   0%|          | 0/38 [00:00<?, ?it/s]

Property 'summary' already exists in node 'c902e0'. Skipping!
Property 'summary' already exists in node '7321a1'. Skipping!
Property 'summary' already exists in node '042f35'. Skipping!
Property 'summary' already exists in node '64468b'. Skipping!
Property 'summary' already exists in node 'c75bca'. Skipping!
Property 'summary' already exists in node '649fd0'. Skipping!
Property 'summary' already exists in node 'e1d5ee'. Skipping!
Property 'summary' already exists in node 'd27994'. Skipping!
Property 'summary' already exists in node 'd5eca0'. Skipping!
Property 'summary' already exists in node 'fe38e0'. Skipping!
Property 'summary' already exists in node '8e45a3'. Skipping!
Property 'summary' already exists in node '9f3b65'. Skipping!
Property 'summary' already exists in node '4b9409'. Skipping!
Property 'summary' already exists in node '318e03'. Skipping!
Property 'summary' already exists in node 'e853b2'. Skipping!
Property 'summary' already exists in node '496676'. Skipping!
Property

...

Property 'summary_embedding' already exists in node 'e1d5ee'. Skipping!
Property 'summary_embedding' already exists in node 'c902e0'. Skipping!
Property 'summary_embedding' already exists in node 'd27994'. Skipping!
Property 'summary_embedding' already exists in node '9f3b65'. Skipping!
Property 'summary_embedding' already exists in node '4b9409'. Skipping!
Property 'summary_embedding' already exists in node '7321a1'. Skipping!
Property 'summary_embedding' already exists in node 'c75bca'. Skipping!
Property 'summary_embedding' already exists in node '64468b'. Skipping!
Property 'summary_embedding' already exists in node '4e4b58'. Skipping!
Property 'summary_embedding' already exists in node '318e03'. Skipping!
Property 'summary_embedding' already exists in node '042f35'. Skipping!
Property 'summary_embedding' already exists in node '649fd0'. Skipping!
Property 'summary_embedding' already exists in node '496676'. Skipping!
Property 'summary_embedding' already exists in node 'd5eca0'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,How many messages were sent weekly by users of...,[Introduction ChatGPT launched in November 202...,"By July 2025, 18 billion messages were being s...",single_hop_specifc_query_synthesizer
1,How do you spell Claude?,[Table 1: ChatGPT daily message counts (millio...,The context indicates that the term is misspel...,single_hop_specifc_query_synthesizer
2,what is Standard Occupation Classification,[Figure 23 presents variation in ChatGPT usage...,Figure 23 shows variation in ChatGPT usage by ...,single_hop_specifc_query_synthesizer
3,how many users are using chatgpt like 700 mill...,[Conclusion This paper studies the rapid growt...,"By July 2025, ChatGPT had been used weekly by ...",single_hop_specifc_query_synthesizer
4,how SOC codes work activities occupation group...,[<1-hop>\n\nFigure 23 presents variation in Ch...,"Figure 23 shows ChatGPT use by occupation, wit...",multi_hop_abstract_query_synthesizer
5,How does the clasification of chatbot messags ...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context explains that ChatGPT, launched in...",multi_hop_abstract_query_synthesizer
6,how message volume compare Jun 2024 to Jun 202...,[<1-hop>\n\nMonth Non-Work (M) (%) Work (M) (%...,Jun 2024 had 451 million total messages with 5...,multi_hop_abstract_query_synthesizer
7,How does ChatGPT usage classify msgs and how i...,[<1-hop>\n\nIntroduction ChatGPT launched in N...,"The context explains that ChatGPT, launched in...",multi_hop_abstract_query_synthesizer
8,Considering the rapid growth of ChatGPT usage ...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context indicates that as of July 2025, ov...",multi_hop_specific_query_synthesizer
9,Wha US is the impact of ChatGPT's growith on n...,[<1-hop>\n\nTable 1: ChatGPT daily message cou...,"The context indicates that as of July 2025, ab...",multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [18]:
from langsmith import Client

client = Client()

dataset_name = "Use Case Synthetic Data - AIE8 - 2.0"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Synthetic Data for Use Cases"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [19]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [20]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [22]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [23]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG"
)

In [24]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [25]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

As is usual: We'll be using `gpt-4.1-mini` for our RAG!

In [26]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [27]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [28]:
rag_chain.invoke({"question" : "What are people doing with AI these days?"})

'Based on the provided context, people are using AI, specifically generative AI like ChatGPT, in a variety of ways including performing workplace tasks that either augment or automate human labor. Users seek to use generative AI both at work and outside of work, producing outputs such as writing, software code, spreadsheets, and other digital products. The usage spans across asking for information or advice, doing tasks (generating outputs), and expressing themselves. \n\nAdditionally, generative AI is flexible and used in many economic tasks, impacting occupational roles and productivity. People also engage with AI not only for professional purposes but for self-expression, relationships, personal reflection, and games or role play, though these uses are less common.\n\nIn summary, people are using AI to:\n\n- Augment or automate workplace tasks  \n- Produce digital content like writing, coding, spreadsheets  \n- Seek information and advice  \n- Enhance productivity and problem-solvin

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [29]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [30]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

dopeness_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "dopeness": "Is this response dope, lit, cool, or is it just a generic response?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

- `qa_evaluator`:
- `labeled_helpfulness_evaluator`:
- `dopeness_evaluator`:

## LangSmith Evaluation

In [31]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'frosty-parent-81' at:
https://smith.langchain.com/o/7ffaf126-290e-4d08-9a81-6ef0b42d5153/datasets/043dd80a-d618-4c22-bc5a-61c3c8c304dd/compare?selectedSessions=6cf89149-69bc-42b1-a0a1-c639c06aad81




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,OpenAI ChatGPT use info how much users use for...,Based on the provided context:\n\n- By July 20...,None,The context shows that since ChatGPT launched ...,1,1,0,6.782197,71fea6b4-28ff-4072-a1db-fea990909b58,2133a436-8fd7-49c0-b191-a9b330ac8e35
1,Hwo do Handa et al. (2025) and Handa et al. (2...,I don't know.,None,"Based on the context, Handa et al. (2025) is a...",0,0,0,1.011096,b0e1f79b-30c8-4c65-9100-125a3d3bcbd4,f199b802-2dda-47df-a0c5-89c95bf5cb14
2,Wha US is the impact of ChatGPT's growith on n...,The context indicates that non-work-related me...,None,"The context indicates that as of July 2025, ab...",1,1,0,7.204578,a2f29471-3325-4636-bb4f-eb38e316af03,bb96157b-b7e4-4905-b1be-7ba91c50a0d2
3,Considering the rapid growth of ChatGPT usage ...,I don't know.,None,"The context indicates that as of July 2025, ov...",0,0,0,0.752724,a6ceafe9-52c3-4825-b1f3-aa6c1fb0327d,ffbc16ea-8b14-49ba-9511-7169295ddfc9
4,How does ChatGPT usage classify msgs and how i...,"Based on the provided context, ChatGPT usage c...",None,"The context explains that ChatGPT, launched in...",1,0,0,5.722868,a00ef836-f923-4d30-9dea-bedaa8639db7,bb175c3d-5734-4aa7-8974-21f3f3b0db0d
5,how message volume compare Jun 2024 to Jun 202...,"Between June 2024 and June 2025, the total dai...",None,Jun 2024 had 451 million total messages with 5...,1,1,0,3.230808,077ce9d6-c7ce-41f2-9e15-4f642288dbd8,59d34b37-6d37-4a5e-85d4-094d9b747387
6,How does the clasification of chatbot messags ...,The classification of chatbot messages is used...,None,"The context explains that ChatGPT, launched in...",1,1,0,5.964357,3891b727-9aad-4d6b-9eb9-f56ee9274213,7473e2cd-b612-4e2e-8a20-5142f8f4bd81
7,how SOC codes work activities occupation group...,"Based on the provided context, SOC (Standard O...",None,"Figure 23 shows ChatGPT use by occupation, wit...",1,1,0,11.413788,aac6f529-92b0-4085-8344-8be0d7c5ec3b,2f3700a2-e661-4c81-a3bd-ea9d038d83c8
8,how many users are using chatgpt like 700 mill...,Based on the provided context from the documen...,None,"By July 2025, ChatGPT had been used weekly by ...",1,1,0,13.009396,7341797c-51a5-4660-a85e-32d68154ac87,46bf8c93-324d-4f8a-92db-62a009048f13
9,what is Standard Occupation Classification,The Standard Occupation Classification (SOC) i...,None,Figure 23 shows variation in ChatGPT usage by ...,1,1,0,1.847627,f70c08f6-a63c-4e7d-b747-ccbce8ed578f,ecf2c5af-950a-4607-9fbd-97c04167e508


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [32]:
DOPENESS_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Make your answer rad, ensure high levels of dopeness. Do not be generic, or give generic responses.

Context: {context}
Question: {question}
"""

dopeness_rag_prompt = ChatPromptTemplate.from_template(DOPENESS_RAG_PROMPT)

In [33]:
rag_documents = docs

In [34]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

In [36]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Use Case RAG Docs"
)

In [37]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [39]:
dopeness_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | dopeness_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [40]:
dopeness_rag_chain.invoke({"question" : "How are people using AI to make money?"})

'Alright, buckle up because the scoop on how people are cashing in with AI is straight fire. Based on the context from the legendary "How People Use ChatGPT," folks aren’t just using AI as a bland tool to tick boxes—they’re flexing AI as their personal advisor and research sidekick. This means that instead of AI just grinding through boring tasks, it’s boosting brainpower, powering decision-making, and stacking up productivity especially in knowledge-heavy gigs.\n\nHere’s the rad part: users are getting paid serious bank because AI isn’t replacing jobs—it’s supercharging the quality of work by leveling up decisions. Think of AI as that ultra-sharp co-pilot that helps knowledge workers slay their projects faster and smarter, turning decision support into cold, hard $$$.\n\nPlus, Collis and Brynjolfsson (2025) threw down some wild numbers: imagine the US alone pulling out $97 billion+ in consumer surplus annually from AI use. That’s the kind of economic mojo showing the AI hustle isn’t j

Finally, we can evaluate the new chain on the same test set!

In [41]:
evaluate(
    dopeness_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        dopeness_evaluator
    ],
    metadata={"revision_id": "dopeness_rag_chain"},
)

View the evaluation results for experiment: 'shiny-lip-97' at:
https://smith.langchain.com/o/7ffaf126-290e-4d08-9a81-6ef0b42d5153/datasets/043dd80a-d618-4c22-bc5a-61c3c8c304dd/compare?selectedSessions=1d2afb06-f52b-4381-ae8e-c2dd2f365d6f




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.dopeness,execution_time,example_id,id
0,OpenAI ChatGPT use info how much users use for...,"Yo, buckle up for some wicked insight straight...",None,The context shows that since ChatGPT launched ...,1,1,1,5.006302,71fea6b4-28ff-4072-a1db-fea990909b58,f489720c-528e-47d5-885f-c957496ee001
1,Hwo do Handa et al. (2025) and Handa et al. (2...,"Yo, let’s crank up the brainwaves and unpack t...",None,"Based on the context, Handa et al. (2025) is a...",0,0,1,4.431902,b0e1f79b-30c8-4c65-9100-125a3d3bcbd4,daf94775-8ca3-4e81-b3cc-87d62ba7b9f8
2,Wha US is the impact of ChatGPT's growith on n...,"Alright, let’s break down the cosmic impact of...",None,"The context indicates that as of July 2025, ab...",1,1,1,9.901098,a2f29471-3325-4636-bb4f-eb38e316af03,5cb4b4f4-0e3c-4a12-8358-804afd742afd
3,Considering the rapid growth of ChatGPT usage ...,"Yo, buckle up because this thing’s dive-deep o...",None,"The context indicates that as of July 2025, ov...",1,1,1,6.506674,a6ceafe9-52c3-4825-b1f3-aa6c1fb0327d,e919626b-1151-41a6-b515-031c501b21a8
4,How does ChatGPT usage classify msgs and how i...,"Alright, strap in for some juicy knowledge str...",None,"The context explains that ChatGPT, launched in...",1,1,1,6.602697,a00ef836-f923-4d30-9dea-bedaa8639db7,a5f6a0d7-4a16-47c9-ac39-ffb7b7310749
5,how message volume compare Jun 2024 to Jun 202...,"Alright, strap in for some next-level knowledg...",None,Jun 2024 had 451 million total messages with 5...,1,1,1,5.617436,077ce9d6-c7ce-41f2-9e15-4f642288dbd8,c49c8747-09ba-4e3e-9240-8d8c817c0724
6,How does the clasification of chatbot messags ...,"Alright, let's crank up the cool factor and br...",None,"The context explains that ChatGPT, launched in...",1,1,1,9.501700,3891b727-9aad-4d6b-9eb9-f56ee9274213,95048054-fb6d-4256-9aa3-9307f993fed5
7,how SOC codes work activities occupation group...,"Alright, let’s drop some knowledge bombs about...",None,"Figure 23 shows ChatGPT use by occupation, wit...",1,1,1,16.835981,aac6f529-92b0-4085-8344-8be0d7c5ec3b,614ca307-e800-4e5e-859b-efe1fc75bcd9
8,how many users are using chatgpt like 700 mill...,"Yo, buckle up for this mind-blowing drip about...",None,"By July 2025, ChatGPT had been used weekly by ...",1,1,1,10.015972,7341797c-51a5-4660-a85e-32d68154ac87,c3cb59c8-6c88-4d91-b683-8597d791667b
9,what is Standard Occupation Classification,"Yo, here’s the skinny on Standard Occupation C...",None,Figure 23 shows variation in ChatGPT usage by ...,1,1,1,2.543294,f70c08f6-a63c-4e7d-b747-ccbce8ed578f,afc82551-40e7-4510-b078-7ef58261c5d8


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.